In [1]:
import utils
loaded_teams = utils.load_teams_from_json("teams.json")

Loaded teams from JSON: [{'code': '18bb7c10', 'name': 'Arsenal'}, {'code': 'b8fd03ef', 'name': 'Manchester-City'}, {'code': '8602292d', 'name': 'Aston-Villa'}, {'code': 'cff3d9bb', 'name': 'Chelsea'}, {'code': '47c64c55', 'name': 'Crystal-Palace'}, {'code': '822bd0ba', 'name': 'Liverpool'}, {'code': '8ef52968', 'name': 'Sunderland'}, {'code': '19538871', 'name': 'Manchester-United'}, {'code': 'd3fd31cc', 'name': 'Everton'}, {'code': 'd07537b9', 'name': 'Brighton-and-Hove-Albion'}, {'code': '361ca564', 'name': 'Tottenham-Hotspur'}, {'code': 'b2b47a98', 'name': 'Newcastle-United'}, {'code': 'fd962109', 'name': 'Fulham'}, {'code': 'cd051869', 'name': 'Brentford'}, {'code': '4ba7cbea', 'name': 'Bournemouth'}, {'code': 'e4a775cb', 'name': 'Nottingham-Forest'}, {'code': '5bfb9659', 'name': 'Leeds-United'}, {'code': '7c21e445', 'name': 'West-Ham-United'}, {'code': '943e8050', 'name': 'Burnley'}, {'code': '8cec06e1', 'name': 'Wolverhampton-Wanderers'}]


In [6]:
#get all the teams name
set_team_names = set()
for team in loaded_teams:
    set_team_names.add(team['name'])
print(set_team_names)

{'Brentford', 'Liverpool', 'Burnley', 'West-Ham-United', 'Aston-Villa', 'Bournemouth', 'Leeds-United', 'Chelsea', 'Manchester-United', 'Sunderland', 'Everton', 'Manchester-City', 'Fulham', 'Newcastle-United', 'Nottingham-Forest', 'Wolverhampton-Wanderers', 'Brighton-and-Hove-Albion', 'Crystal-Palace', 'Arsenal', 'Tottenham-Hotspur'}


In [2]:
all_matches = utils.get_all_season_match(season="2023-2024")

Loaded teams from JSON: [{'code': '18bb7c10', 'name': 'Arsenal'}, {'code': 'b8fd03ef', 'name': 'Manchester-City'}, {'code': '8602292d', 'name': 'Aston-Villa'}, {'code': 'cff3d9bb', 'name': 'Chelsea'}, {'code': '47c64c55', 'name': 'Crystal-Palace'}, {'code': '822bd0ba', 'name': 'Liverpool'}, {'code': '8ef52968', 'name': 'Sunderland'}, {'code': '19538871', 'name': 'Manchester-United'}, {'code': 'd3fd31cc', 'name': 'Everton'}, {'code': 'd07537b9', 'name': 'Brighton-and-Hove-Albion'}, {'code': '361ca564', 'name': 'Tottenham-Hotspur'}, {'code': 'b2b47a98', 'name': 'Newcastle-United'}, {'code': 'fd962109', 'name': 'Fulham'}, {'code': 'cd051869', 'name': 'Brentford'}, {'code': '4ba7cbea', 'name': 'Bournemouth'}, {'code': 'e4a775cb', 'name': 'Nottingham-Forest'}, {'code': '5bfb9659', 'name': 'Leeds-United'}, {'code': '7c21e445', 'name': 'West-Ham-United'}, {'code': '943e8050', 'name': 'Burnley'}, {'code': '8cec06e1', 'name': 'Wolverhampton-Wanderers'}]


In [3]:
print(len(all_matches))
all_matches[:5]

630


['/en/matches/15addfc7/Everton-Fulham-August-12-2023-Premier-League',
 '/en/matches/2d364b37/Sunderland-Norwich-City-October-28-2023-Championship',
 '/en/matches/e2c53b0a/Burnley-Sheffield-United-December-2-2023-Premier-League',
 '/en/matches/eb7c6993/Brighton-and-Hove-Albion-Everton-February-24-2024-Premier-League',
 '/en/matches/e9d5dd9b/Crystal-Palace-Brentford-December-30-2023-Premier-League']

In [11]:
import re
from datetime import datetime
import pandas as pd

def parse_match_url(url):
    """
    Parses a match URL to extract only the unique match_id and event_time,
    using a non-greedy approach for the date part to ensure the full month name is captured.
    """
    
    # 1. Regex to capture the ID and the date part from the end of the string.
    # The date is typically followed by the league name (e.g., '-Premier-League').
    # We look for the ID, and then the full date pattern near the end.
    
    # Pattern explanation:
    # 1. /en/matches/([\da-fA-F]+)/ - Capture the unique ID.
    # 2. .*? - Non-greedily match everything until...
    # 3. (\w+-\d{2}-\d{4}) - Capture the Month-Day-Year pattern (this is the date part).
    
    pattern = r'/en/matches/([\da-fA-F]+)/.*?(\w+-\d{2}-\d{4})'
    match_data = re.search(pattern, url)
    
    if not match_data:
        pattern = r'/en/matches/([\da-fA-F]+)/.*?(\w+-\d{1}-\d{4})'
        match_data = re.search(pattern, url)

        if not match_data:
            print(f"Error: URL structure mismatch for: {url}")
            return None
        

    unique_match_id = match_data.group(1)
    # This group should now capture the full month name (e.g., 'August-12-2023')
    date_part = match_data.group(2) 

    # --- Convert Date ---
    try:
        # Format string remains '%B-%d-%Y' as it expects the full month name (e.g., "August")
        date_obj = datetime.strptime(date_part, '%B-%d-%Y')
        event_time = pd.to_datetime(date_obj.strftime('%Y-%m-%d'))
    except ValueError as e:
        # You will still see this error if the data source provided a truncated month,
        # but the corrected regex should reduce it significantly.
        print(f"Error parsing date: {date_part}") 
        return None
    
    return {
        'match_id': unique_match_id, 
        'event_time': event_time
    }

In [ ]:
import pandas as pd
import numpy as np
import time
# Assuming utils.stat_player_match and all_matches (list of URLs) are available

full_player_stats = pd.DataFrame()

# Helper function to create the player hash ID
def create_player_hash_id(df):
    if 'Player' not in df.columns:
         # Handle case where player name is split or missing
         return np.nan 
    
    # Standardize the name and create a robust hash ID
    standardized_name = df['Player'].astype(str).str.strip().str.lower()
    # Use a modulo for a positive integer hash suitable for a key
    return standardized_name.apply(lambda x: hash(x) % (2**31 - 1)) 

for match_url in all_matches:
    time.sleep(5)   
    # --- Step 1: Extract Minimal Metadata from URL ---
    match_data = parse_match_url(match_url)
    
    if not match_data:
        # print(f"Skipping URL due to parsing error: {match_url}")
        continue
        
    event_time = match_data['event_time']
    match_id = match_data['match_id']
    
    # --- Step 2: Get Player Stats ---
    try:
        df1, df2 = utils.stat_player_match(match_url) 
    except Exception as e:
        # print(f"Error fetching stats for {match_url}: {e}")
        continue # Skip to the next match upon error

    # --- Step 3: Clean and Prepare DataFrames ---
    df1 = df1.fillna(0)
    df2 = df2.fillna(0)
    
    # --- Step 4: Add Metadata and Player IDs ---
    
    # Team 1 (df1)
    df1['match_id'] = match_id
    df1['event_time'] = event_time
    df1['team_indicator'] = 1 # Team 1
    df1['player_id'] = create_player_hash_id(df1)

    # Team 2 (df2)
    df2['match_id'] = match_id
    df2['event_time'] = event_time
    df2['team_indicator'] = 2 # Team 2
    df2['player_id'] = create_player_hash_id(df2)

    # 5. Combine and Append
    match_df = pd.concat([df1, df2], ignore_index=True)
    full_player_stats = pd.concat([full_player_stats, match_df], ignore_index=True)


# 6. Final Clean-up
full_player_stats['player_id'] = full_player_stats['player_id'].astype('int64')
full_player_stats['team_indicator'] = full_player_stats['team_indicator'].astype('int8')
# match_id is the unique string ID part



In [13]:
print(full_player_stats.head())
print(f"Total player-match observations: {len(full_player_stats)}")

               Player     #  Nation    Pos     Age  Min  Performance_Gls  \
0         Neal Maupay  13.0  fr FRA     FW  26-363   71                0   
1      Arnaut Danjuma  10.0  nl NED     FW  26-193   19                0   
2  Abdoulaye Doucouré  16.0  ml MLI     FW  30-223   90                0   
3          Alex Iwobi  17.0  ng NGA  LM,RM  27-101   90                0   
4        Amadou Onana   8.0  be BEL     CM  21-361   90                0   

   Performance_Ast  Performance_PK  Performance_PKatt  ...  team_indicator  \
0                0               0                  0  ...               1   
1                0               0                  0  ...               1   
2                0               0                  0  ...               1   
3                0               0                  0  ...               1   
4                0               0                  0  ...               1   

    player_id  Performance_Fls  Performance_Fld  Performance_Off  \
0   91

In [14]:
full_player_stats.to_csv("data/full_player_stats_2023_2024.csv", index=False)

In [16]:
#remove the columns after player_id
print(full_player_stats.columns)
new_columns = full_player_stats.columns.tolist()[:full_player_stats.columns.tolist().index('player_id') + 1]
full_player_stats = full_player_stats[new_columns]

Index(['Player', '#', 'Nation', 'Pos', 'Age', 'Min', 'Performance_Gls',
       'Performance_Ast', 'Performance_PK', 'Performance_PKatt',
       'Performance_Sh', 'Performance_SoT', 'Performance_CrdY',
       'Performance_CrdR', 'Performance_Touches', 'Performance_Tkl',
       'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG',
       'Expected_xAG', 'SCA_SCA', 'SCA_GCA', 'Passes_Cmp', 'Passes_Att',
       'Passes_CmpPct', 'Passes_PrgP', 'Carries_Carries', 'Carries_PrgC',
       'Take-Ons_Att', 'Take-Ons_Succ', 'match_id', 'event_time',
       'team_indicator', 'player_id', 'Performance_Fls', 'Performance_Fld',
       'Performance_Off', 'Performance_Crs', 'Performance_TklW',
       'Performance_OG', 'Performance_PKwon', 'Performance_PKcon'],
      dtype='object')


In [17]:
full_player_stats.to_csv("data/full_player_stats_2023_2024.csv", index=False)

In [18]:
print(full_player_stats.columns)

Index(['Player', '#', 'Nation', 'Pos', 'Age', 'Min', 'Performance_Gls',
       'Performance_Ast', 'Performance_PK', 'Performance_PKatt',
       'Performance_Sh', 'Performance_SoT', 'Performance_CrdY',
       'Performance_CrdR', 'Performance_Touches', 'Performance_Tkl',
       'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG',
       'Expected_xAG', 'SCA_SCA', 'SCA_GCA', 'Passes_Cmp', 'Passes_Att',
       'Passes_CmpPct', 'Passes_PrgP', 'Carries_Carries', 'Carries_PrgC',
       'Take-Ons_Att', 'Take-Ons_Succ', 'match_id', 'event_time',
       'team_indicator', 'player_id'],
      dtype='object')


## Rolling average

In [20]:
import pandas as pd

ROLLING_WINDOW_SIZE = 10
# Ensure data is sorted for correct rolling calculation
df_player_stats = full_player_stats.sort_values(['player_id', 'event_time'])

# Define the Rolling Window Function
# window=6 means 6 rows total. The .shift(1) handles excluding the current row.
# The window includes the 5 matches PRECEDING the current one.
def calculate_rolling_avg(series):
    # This calculates a 10-match rolling sum, then shifts it to align with the *next* match.
    # We use a window size of 10 for the aggregation.
    return series.rolling(window=ROLLING_WINDOW_SIZE, min_periods=1).mean().shift(1)

columns_for_rolling =full_player_stats.columns.tolist()[full_player_stats.columns.tolist().index('Min'):full_player_stats.columns.tolist().index('Take-Ons_Succ') + 1]

for col in columns_for_rolling:
    df_player_stats[f'rolling_avg_{col}'] = df_player_stats.groupby('player_id')[col].transform(calculate_rolling_avg)
    df_player_stats[f'rolling_avg_{col}'] = df_player_stats[f'rolling_avg_{col}'].fillna(0)

In [21]:
print(df_player_stats.columns)

Index(['Player', '#', 'Nation', 'Pos', 'Age', 'Min', 'Performance_Gls',
       'Performance_Ast', 'Performance_PK', 'Performance_PKatt',
       'Performance_Sh', 'Performance_SoT', 'Performance_CrdY',
       'Performance_CrdR', 'Performance_Touches', 'Performance_Tkl',
       'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG',
       'Expected_xAG', 'SCA_SCA', 'SCA_GCA', 'Passes_Cmp', 'Passes_Att',
       'Passes_CmpPct', 'Passes_PrgP', 'Carries_Carries', 'Carries_PrgC',
       'Take-Ons_Att', 'Take-Ons_Succ', 'match_id', 'event_time',
       'team_indicator', 'player_id', 'rolling_avg_Min',
       'rolling_avg_Performance_Gls', 'rolling_avg_Performance_Ast',
       'rolling_avg_Performance_PK', 'rolling_avg_Performance_PKatt',
       'rolling_avg_Performance_Sh', 'rolling_avg_Performance_SoT',
       'rolling_avg_Performance_CrdY', 'rolling_avg_Performance_CrdR',
       'rolling_avg_Performance_Touches', 'rolling_avg_Performance_Tkl',
       'rolling_avg_Performan

In [22]:
df_player_stats.to_csv("data/full_player_stats_rollings_2023_2024.csv", index=False)

In [1]:
import pandas as pd
if True:
    df_player_stats = pd.read_csv('data/player_stats_with_rolling_avgs.csv')

In [ ]:
#save to hopsworks feature store
import hopsworks
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key from .env
api_key = os.getenv("HOPSWORKS_API_KEY")

if not api_key:
    raise ValueError("HOPSWORKS_API_KEY not found in .env file")

# Connect to Hopsworks
project = hopsworks.login(
    project='player_stat_prediction', 
    api_key_value=api_key,
    host="eu-west.cloud.hopsworks.ai",
    port=443
)

# Get the feature store
fs = project.get_feature_store()

# Create or get the feature group for player stats with rolling averages
fg = fs.get_or_create_feature_group(
    name="player_stats_rolling",
    version=1,
    primary_key=["player_id", "match_id"],
    event_time="event_time",
    online_enabled=True,
    stream=False,
    description="Player statistics with rolling averages for all season"
)

# Insert the data into the feature group
fg.insert(df_player_stats, write_options={"wait_for_job": True})

print("Data successfully saved to Hopsworks feature store!")
print(f"Feature group '{fg.name}' version {fg.version} has been updated.")

2026-01-05 19:24:05,659 INFO: Initializing external client
2026-01-05 19:24:05,660 INFO: Base URL: https://c.app.hopsworks.ai:443


/home/hugo/projects/player_stats_predictor/footenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2026-01-05 19:24:06,181 INFO: Closing external client and cleaning up certificates.
2026-01-05 19:24:06,186 INFO: Connection closed.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project). Server response: 
HTTP code: 401, HTTP reason: Unauthorized, body: b'{"errorCode":320013,"errorMsg":"API key not found in the database"}', error code: 320013, error msg: API key not found in the database, user msg: 